# Mixed-Integer Linear Programming (MILP) for Local Interpretable Model-agnostic Explanations (LIME)

This study aims to formulate (and test) Local Interpretable Model-agnostic Explanations (LIME) using Mixed-Integer Linear Programming (MILP).

- Lucas Emanuel Resck Domingues
- Professor: Luciano Guimarães
- FGV-EMAp

## Introduction

The Optimization field is today one of the most important fields of Applied Mathematics.
Not only the theory and the research are very solid, but also the applications are everywhere, from Engineering to Management. In general, problems of this kind try to deal with the maximization or minimization of a function, given some conditions in the variables.

The main problem of Machine Learning, on the other hand, is to search for a function that represents the observed phenomenon using observed data. In fact, it can also be seen as an optimization problem, where the optimal function (given the data) is sought. For example, the goal of training a deep neural network is to search for a local mininum (with luck, a global mininum) for the loss function, given the data.

Machine Learning models have a problem: many of them are not interpretable. Some of them are so incognito that they are called "black-box", in reference to airplane's black-box.
This way, many researchers presented methods of explaining and interpreting Machine Learning model results.
One of them is Local Interpretable Model-agnostic Explanations (LIME), the focus of this study.

Suppose you have a text classification model (each text is assigned to a class) that says you if a text is mathematical or not. You want to understand why the model says this Introduction text (we call it $x$) is mathematical. LIME disturbs the text $x$ and verify the changes in probabilities $f(x)$. It tries to fit a simpler (but interpretable) model $g$, trying to predict the probabilities $f(x)$ over the pertubations of the input $x$. Mathematically, LIME tries to search for

$$\xi(x) = \mathrm{argmin}_{g \in G} \mathcal{L}(f, g, \pi_x) + \Omega(g),$$

being $G$ the set of desired interpretable functions, $\pi_x$ the locallity around $x$, $\mathcal{L}$ some measure of non proximity between $f, g$ in the locallity given by $\pi_x$, and $\Omega(g)$ being the complexity of $g$.
Very abstract, but it will be more clear in the following sections.

In general, LIME formulation is not MILP, but we can addapt the formulation to have it MILP.

## Setup

Only Python and Jupyter stuff. You can jump over this section.

In [ ]:
from IPython.display import HTML
from pulp import LpVariable, LpProblem, value, LpStatus, LpMinimize
from sklearn.calibration import CalibratedClassifierCV
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
import numpy as np
import pandas as pd
import string

In [ ]:
%config Completer.use_jedi = False

## Model

### Data

In [ ]:
df = pd.read_csv('../data/IMDB Dataset.csv')
df

In [ ]:
def preprocessing(text):
    '''Preprocess text.'''
    return text.replace('<br />', '')

In [ ]:
df.review = df.review.apply(preprocessing)

In [ ]:
X = df.review.to_list()
y = df.sentiment.to_list()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

### Classifier

In [ ]:
tf_idf = TfidfVectorizer(
    strip_accents=None,
    lowercase=True,
    smooth_idf=True,
)
X_train = tf_idf.fit_transform(X_train)
X_train.shape

In [ ]:
t_svd = TruncatedSVD(n_components=50, random_state=42)
X_train = t_svd.fit_transform(X_train)
X_train.shape

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

In [ ]:
estimator = LinearSVC(
    dual=False,  # n_samples > n_features
    verbose=1,
    random_state=42
)

svm = GridSearchCV(
    estimator,
    param_grid={'C': np.logspace(-2, 10, 13)},
    cv=5,
    n_jobs=-1,
    verbose=3
)

In [ ]:
%%time
svm.fit(X_train, y_train)

In [ ]:
%%time
svm = CalibratedClassifierCV(svm.best_estimator_, cv=5)
svm.fit(X_train, y_train)

In [ ]:
vector = Pipeline([
    ('tf_idf', tf_idf),
    ('t_svd', t_svd)
])

In [ ]:
model = Pipeline([
    ('tf_idf', tf_idf),
    ('t_svd', t_svd),
    ('scaler', scaler),
    ('svm', svm)
])

In [ ]:
model.score(X_test, y_test)

## Optimization

### LIME

### Calculation of parameters

In [ ]:
def f(text):
    '''Probability of a text'''
    return model.predict_proba([text])[0]

In [ ]:
def pi(x, z, sigma_2=-1/np.log(0.1)):
    '''Weights of locallity.'''
    x = vector.transform([x])[0]
    z = vector.transform([z])[0]
    # If null vector
    if np.abs(z).sum() == 0:
        return 0
    # Cosine of angle between vectors
    cos = np.dot(x, z)/(np.linalg.norm(x)*np.linalg.norm(z))
    # If cosine is like 1.00008
    if cos > 1:
        cos = 1
    # Angle between vectors, normalized to between 0 and 1
    D = np.arccos(cos)*2/np.pi
    return np.exp(-D**2/sigma_2)

In [ ]:
def parameters(split, which_class, M, N, K):
    '''Return the parameters for LIME optimization.'''
    # Perturbations
    z_line = []
    # Probabilities
    f_z = []
    # Weights
    pi_x = []
    
    for i in range(N):
        # Choose a random number of words to remove, between 1 and (split - 1)
        n = np.random.choice(range(1, M))
        # Remove n random words
        indices = np.random.choice(range(M), size=n, replace=False)
        
        # The pertubartion
        perturbation = np.ones(M)
        for index in indices:
            perturbation[index] = 0
        z_line.append(perturbation)
            
        # The probability and weight
        text = ' '.join([word for (j, word) in enumerate(split) if perturbation[j]])
        f_z.append(f(text)[which_class])
        pi_x.append(pi(example, text))
    
    return z_line, f_z, pi_x

### Linear optimization

In [ ]:
def optimization(z_line, f_z, pi_x, M, N, K, lambda_=0.5):
    '''The MILP for LIME.'''
    prob = LpProblem("LIME", LpMinimize)
    
    # Variables
    L = LpVariable('L')
    Omega = LpVariable('Omega')
    epsilon = [LpVariable('epsilon_{}'.format(i)) for i in range(N)]
    delta = [LpVariable('delta_{}'.format(i)) for i in range(M)]
    g = [LpVariable("g(z'_{})".format(i)) for i in range(N)]
    x = [LpVariable('x_{}'.format(j)) for j in range(M)]
#     y = [LpVariable('y_{}'.format(j), 0, 1, cat='Integer') for j in range(M)]
    
    # Objective
    prob += L + lambda_*Omega
    
    # Constraints
    prob += L == sum([pi_x[i]*epsilon[i] for i in range(N)])
    prob += Omega == sum(delta)

    for i in range(N):
        prob += -epsilon[i] <= f_z[i] - g[i]
        prob += epsilon[i] >= f_z[i] - g[i]
        prob += g[i] == sum([z_line[i][j]*x[j] for j in range(M)])

    infinity = 100000
    for j in range(M):
        prob += -delta[j] <= x[j]
        prob += delta[j] >= x[j]
        
#         prob += -infinity*y[j] <= x[j]
#         prob += infinity*y[j] >= x[j]

#     prob += sum(y) <= K

    print('Solving MILP...')
    status = prob.solve()
    print('Done.')
    
    return prob, status, x

In [ ]:
def visualize(split, importances):
    '''Visualize the importance of each word in the classification.'''
    max_abs_importance = np.max(np.abs(importances))
    # Green
    positive = np.array([0, 255, 0])
    white = np.array([255, 255, 255])
    # Red
    negative = np.array([255, 0, 0])
    spans = []
    for i, word in enumerate(split):
        if importances[i] >= 0:
            color = white + (positive - white)/max_abs_importance*importances[i]
        else:
            color = white + (negative - white)/((-1)*max_abs_importance)*importances[i]
        spans.append(
            '<span style="background-color: RGB({R}, {G}, {B})">{word}</span>'.format(
                word=word,
                R=color[0],
                G=color[1],
                B=color[2]
            )
        )
            
    html = ' '.join(spans)
    return HTML(html)

In [ ]:
def lime(text, which_class, N=None, K=None):
    # Split 
    split = text.split()
    M = len(split)
    if N is None:
        N = 5*M
#     if K is None:
#         K = min([M, 20])
                
    z_line, f_z, pi_x = parameters(split, which_class, M, N, K)
        
    prob, status, x = optimization(z_line, f_z, pi_x, M, N, K)
    
    importances = [value(i) for i in x]    
    print(dict(zip(split, importances)))    
    
    return visualize(split, importances)

### Examples

In [ ]:
example = 'This movie is awful, I regret seing it, it is a bad movie.'
example

In [ ]:
model.predict([example])

In [ ]:
model.predict_proba([example])

In [ ]:
lime(example, 0)

## References

- https://arxiv.org/pdf/1602.04938.pdf
- https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
- https://vanderbei.princeton.edu/tex/talks/MOPTA14/L1_reg.pdf